# 🔐 RAG + Guardrails Chatbot

Этот ноутбук демонстрирует Retrieval-Augmented Generation (RAG) с предобученной моделью (DialoGPT), безопасностью (guardrails), фильтрацией логитов и логированием.

In [ ]:
# 📦 Импорт библиотек
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import faiss
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import random
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

In [ ]:
# 🔎 FAISS + RAG: индексация и поиск
corpus = [
    "Диалог — важная форма общения между людьми и машинами.",
    "Языковые модели помогают в создании интеллектуальных ассистентов.",
    "Безопасность — ключевой аспект при использовании ИИ.",
    "RAG объединяет извлечение и генерацию текста."
]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus).toarray().astype('float32')
index = faiss.IndexFlatL2(X.shape[1])
index.add(X)

def retrieve(query):
    q_vec = vectorizer.transform([query]).toarray().astype('float32')
    D, I = index.search(q_vec, k=1)
    return corpus[I[0][0]]

In [ ]:
# 🤖 Загрузка модели
model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
# 🚫 Guardrails — фильтрация запрещённых слов
blocklist = ["бомба", "террор", "убить"]

def is_safe(text):
    return not any(bad in text.lower() for bad in blocklist)

In [ ]:
# 🔡 Генерация ответа
def generate_response(prompt, chat_history_ids=None):
    input_text = retrieve(prompt) + " " + prompt
    input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors='pt')
    if chat_history_ids is not None:
        input_ids = torch.cat([chat_history_ids, input_ids], dim=-1)
    output_ids = model.generate(input_ids, max_length=200, pad_token_id=tokenizer.eos_token_id)
    reply = tokenizer.decode(output_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    return output_ids, reply

In [ ]:
# 🧪 Пример использования
chat_history = None
log_data = []

for user_input in ["Что такое RAG?", "Как работает диалоговая модель?"]:
    chat_history, response = generate_response(user_input, chat_history)
    safe = is_safe(response)
    log_data.append({"query": user_input, "response": response, "safe": safe})
    print(f"Пользователь: {user_input}\nБОТ: {response}\n{'-'*50}")

In [ ]:
# 📊 Анализ логов
df = pd.DataFrame(log_data)
df